# Define Functions and Run Initial Query

In [33]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

def read_query_file(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

In [41]:
run_query("""

WITH
DateSelector As (
    /*
    Builds a table with min and max dates.
    Kind of overengineered but it prevents dates
    getting changed in one place but not another.
    */
    SELECT *
    FROM
        ( VALUES (CAST('2023-07-01' AS DATE), CAST('2023-08-31' AS DATE)) )
        as t ("StartDate","EndDate")
)
SELECT
    CAST(eh.expert_id AS BIGINT) as "Employee",
    eh.tenure_group as "TenureGroup",
    *
FROM
    hive.care.l3_asurion_whole_home_expert_hierarchy eh
WHERE 1=1
    --AND UPPER(eh.business_unit) = 'PSS'
    --AND eh.location = 'flor'
    AND (SELECT MAX(EndDate) FROM DateSelector)
        BETWEEN eh.eff_start_dt and eh.eff_end_dt
    AND (
          (
          eh.supervisor_full_name = 'eddie johnson'
            or 
          (eh.supervisor_full_name like '%brien')
          )
        
        OR
          (
          eh.manager_full_name = 'eddie johnson'
            or 
          (eh.manager_full_name like '%brien' and eh.manager_full_name ='vincent')
          )
        )
""").value_counts('supervisor_full_name')

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_34620\529532665.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


supervisor_full_name
eddie johnson      16
vincent o'brien    12
jason o'brien       3
dtype: int64

In [3]:
df = run_query(read_query_file('SQL\SamplingTest\KeyMetrics_ForGroupSelection.SQL'))
display(df.head())
df['SP100'] = df['Total_Accepted']/df['Total_Eligible']
df['CXP_Score']=df['CXP_Prob_Sum']/df['CXP_Prob_Count']
df['Sessions Per Hour'] = df['Sessions']/df['HoursWorked']
df['CRT']= df['Total_Resolution_Time']/df['Sessions']
df['Searches Per Session']= df['Helix_Searches']/df['Sessions']
df['%Sessions With Search']= df['Helix_Sessions']/df['Sessions']
df['TenureGroup']= df['TenureGroup'].replace({'0-30':'<90','31-60':'<90','61-90':'<90', '91-120':'91-180','121-180':'91-180',})
df=df.join(pd.get_dummies(df['TenureGroup']))
display(df.head())

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_29416\971617532.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,TenureGroup,Date,HoursWorked,Sessions,Total_Resolution_Time,Total_Accepted,Total_Eligible,Helix_Searches,Helix_Sessions,CXP_Prob_Sum,CXP_Prob_Count
0,590730,31-60,2023-08-22,8.11,18.00,"15,571.00",0.00,17.00,3,2,10.74,21.00
1,590730,31-60,2023-09-03,8.08,21.00,"13,127.00",0.00,14.00,1,1,7.13,14.00
2,590730,31-60,2023-08-23,7.44,8.00,"7,615.00",0.00,7.00,5,1,6.57,13.00
3,590730,31-60,2023-08-16,9.20,2.00,"3,390.00",0.00,2.00,9,3,2.08,4.00
4,590730,31-60,2023-08-24,8.85,11.00,"10,101.00",0.00,9.00,3,2,6.18,12.00


,Employee,TenureGroup,Date,HoursWorked,Sessions,Total_Resolution_Time,Total_Accepted,Total_Eligible,Helix_Searches,Helix_Sessions,CXP_Prob_Sum,CXP_Prob_Count,SP100,CXP_Score,Sessions Per Hour,CRT,Searches Per Session,%Sessions With Search,180+,91-180,<90
0,590730,<90,2023-08-22,8.11,18.00,"15,571.00",0.00,17.00,3,2,10.74,21.00,0.00,0.51,2.22,865.06,0.17,0.11,0,0,1
1,590730,<90,2023-09-03,8.08,21.00,"13,127.00",0.00,14.00,1,1,7.13,14.00,0.00,0.51,2.60,625.10,0.05,0.05,0,0,1
2,590730,<90,2023-08-23,7.44,8.00,"7,615.00",0.00,7.00,5,1,6.57,13.00,0.00,0.51,1.08,951.88,0.62,0.12,0,0,1
3,590730,<90,2023-08-16,9.20,2.00,"3,390.00",0.00,2.00,9,3,2.08,4.00,0.00,0.52,0.22,"1,695.00",4.50,1.50,0,0,1
4,590730,<90,2023-08-24,8.85,11.00,"10,101.00",0.00,9.00,3,2,6.18,12.00,0.00,0.51,1.24,918.27,0.27,0.18,0,0,1


# Select Sample Size by Tenure Group
Using the current number of experts find the number of samples needed in each group to be representative of the general population at Orlando

In [5]:
# df = df[~df['Employee'].isin([571644,267315])]
employees_df = df[['Employee','TenureGroup']].copy().drop_duplicates(subset='Employee')

SAMPLE_SIZE = 15
sample_size_df=pd.pivot_table(df.copy(),index='TenureGroup',values='Employee',aggfunc=lambda x: len(x.unique()),margins=True,)
sample_size_df['Total Employees']=sample_size_df['Employee']
del sample_size_df['Employee']
sample_size_df['% of Employees'] = sample_size_df['Total Employees']/sample_size_df['Total Employees'].loc['All']
sample_size_df['n Employees'] = (sample_size_df['% of Employees']* SAMPLE_SIZE).round()
display(sample_size_df)
sample_size_df=sample_size_df.iloc[:-1]
display(sample_size_df.sum())
display(sample_size_df)

del SAMPLE_SIZE, employees_df

,Total Employees,% of Employees,n Employees
TenureGroup,,,
180+,162,0.43,7.00
91-180,100,0.27,4.00
<90,111,0.30,4.00
All,373,1.00,15.00


Total Employees   373.00
% of Employees      1.00
n Employees        15.00
dtype: float64

,Total Employees,% of Employees,n Employees
TenureGroup,,,
180+,162,0.43,7.00
91-180,100,0.27,4.00
<90,111,0.30,4.00


# Test
1. Creates representative samples
2. Tests those samples to see if they have a meaningful effect on the metric
    1. Runs an OLS on the sample to see if they have high p val for the metric. If they have a low value for all metrics they are saved as a good possible sample.

In [ ]:
SCORE_CEIL = .175
SAMPLES = 1000
SelectionMetrics = ['Sessions','Sessions Per Hour','CRT','Searches Per Session','%Sessions With Search','Total_Accepted','SP100']
employees_df = df[['Employee','TenureGroup']].copy().drop_duplicates(subset='Employee')
results_df = pd.DataFrame(columns=['Members','Tenure Group']+SelectionMetrics)

# Build test dataframe dft and get groups setup
groups = sample_size_df.index.to_list()
# df = df.copy()
df['Constant']=1
df = df.replace({np.inf : np.nan})
# Drop some outliers
df = df[df['%Sessions With Search']<1]
df = df[df['Searches Per Session']<2.5]
# px.histogram(df['Searches Per Session']).show()

i=0
current_group = '61-90'
tenure_group_size = 2

group_len = str(len(groups))

for tenure_group in [current_group]:
    i = i+1
    print(timestamp()+" Running "+str(i)+" of "+group_len)
    # Make a loop to run this a bunch
    dft= df[df[tenure_group]==1].copy()
    dft=dft[['Employee','Constant']+SelectionMetrics+groups].copy()
    dft= dft.dropna()
    meanList = []
    stdList = []
    for metric in SelectionMetrics:
        meanList= meanList+[dft[metric].mean()]
        stdList= stdList+[dft[metric].std()]  
    meanDict = dict(zip(SelectionMetrics,meanList))
    stdDict = dict(zip(SelectionMetrics,stdList))
    del meanList, stdList
    for run in range(1,SAMPLES):
        # creates a list for the sampled employes
        # then adds the correct number of samples from each group to it
        sample_group = []
        sample_df = employees_df[employees_df['TenureGroup']==tenure_group]
        sample_group= sample_group+ sample_df['Employee'].sample(
            n=int(tenure_group_size)
            ,replace=False
            # ,random_state=run
            ).tolist()
        
        # Flag the employees in the group
        dft['TestSample']=(dft['Employee'].isin(sample_group)).astype(int)
        
        # See if its worth the regressions
        score_list = []
        score_count = 0
        test_means_df = dft[dft['TestSample']==1].copy()
        for y in SelectionMetrics:
            if SCORE_CEIL >= abs((test_means_df[y].mean()-meanDict[y])/stdDict[y]):
                score_count = score_count+1
            del y
        del test_means_df

        # Run the regresssions
        if score_count == len(SelectionMetrics):
            for y in SelectionMetrics:        
                model = sm.OLS(endog=dft[y].copy(),exog=dft.copy()[['TestSample','Constant']]).fit().summary2().tables[1]
                score = (model['Coef.'].loc['TestSample'])/stdDict[y]
                lower = model['[0.025'].loc['TestSample']
                upper = model['0.975]'].loc['TestSample']
                if abs(score) <= SCORE_CEIL and lower<0 and upper>0:
                    score_list.append(score)
                del score, y
            
        # add sample group to results list if the results were not significant
        if len(score_list)==len(SelectionMetrics):
            current_result = dict(zip(['Members','Tenure Group']+SelectionMetrics,[[sample_group]]+[tenure_group]+score_list))
            # display(current_result)
            results_df=pd.concat([results_df,pd.DataFrame(current_result,index=[run])])
        del sample_group, score_list, dft['TestSample'], sample_df

# del dft, SAMPLES, SCORE_CEIL, run, employees_df, group_len, groups, i, lower, upper, metric,model,meanDict

results_df

# Find and display subsets that do not have any duplicate members

In [ ]:
results_df['TotalSCORE']=0
results_dfp = results_df.copy()
groups = sample_size_df.index.to_list()



i=0
# groups.remove('180+')
# groups.remove('0-30')
# groups.remove('91-120')

for tenure_group in [current_group]:
    results_df = results_dfp[results_dfp['Tenure Group']==tenure_group].copy()
    for metric in SelectionMetrics:
        results_df['TotalSCORE'] = results_df['TotalSCORE']+abs(results_df[metric])
    results_df['AvgSCORE']= results_df['TotalSCORE']/len(SelectionMetrics)
    results_df = results_df.sort_values('AvgSCORE')
    results_df = results_df.drop_duplicates(subset='Members')
    display(results_df.head(20))
    # del metric
    # for row1 in range(len(results_df['Members'])):
    #     for row2 in range(1,len(results_df['Members'])):
    #         duplicates = 0           
    #         for item in results_df['Members'].iloc[row1]:
    #             if item in results_df['Members'].iloc[row2]: 
    #                 duplicates=duplicates+1
    #                 # display(item)
    #         if duplicates == 0:
    #             summary=pd.concat([results_df.iloc[[row1]],results_df.iloc[[row2]]])
    #             display(summary)
results_df = results_dfp.copy()          
print(1)
# results_df.to_excel('Unbiased Sample Selections. Scored 180+.xlsx')